<a href="https://colab.research.google.com/github/MaggieHDez/MathStatsAI/blob/main/factorizacionMatrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Factorizaciones de Matrices y Matriz Diagonalmente Dominante

En este notebook se realiza lo siguiente:

1. **Ingreso de la matriz A:** Se solicita al usuario que ingrese una matriz cuadrada **A**. Cada fila se introduce como una lista de números separados por espacios.
2. **Verificación de diagonalidad:** Se verifica si **A** es diagonalmente dominante. Recordemos que una matriz es diagonalmente dominante si, para cada fila, el valor absoluto del elemento en la diagonal es mayor o igual a la suma de los valores absolutos de los demás elementos de esa misma fila.
3. **Obtención de la matriz B:** Si **A** no es diagonalmente dominante, se busca una permutación de sus filas que genere una matriz **B** con dicha propiedad.
4. **Factorizaciones:** Una vez obtenida la matriz **B**, se realizan tres tipos de factorizaciones:
   - **LU:** Se obtiene la factorización en matrices triangular inferior (**L**) y superior (**U**) utilizando una matriz de permutación (**P**).
   - **QR:** Se descompone **B** en una matriz ortogonal (**Q**) y una matriz triangular superior (**R**).
   - **SVD:** Se realiza la descomposición en valores singulares, obteniendo las matrices **U**, los valores singulares **s** y la matriz transpuesta de **V** (**Vh**).

Los resultados de cada factorización se muestran en pantalla.

In [3]:
import numpy as np
from itertools import permutations
from scipy.linalg import lu

def is_diagonally_dominant(M):

  # Verifica si la matriz M es diagonalmente dominante.
  # Para cada fila, el valor absoluto del elemento diagonal debe ser
  # mayor o igual a la suma de los valores absolutos de los otros elementos de esa fila.

  n = M.shape[0]
  for i in range(n):
    # Suma de los elementos fuera de la diagonal en la fila i
    off_diag_sum = np.sum(np.abs(M[i])) - abs(M[i, i])
    # Compara el valor absoluto del elemento diagonal con la suma anterior
    if abs(M[i, i]) < off_diag_sum:
      return False
  return True

def make_diagonally_dominant(A):

  # Intenta obtener una matriz diagonalmente dominante a partir de A
  # probando todas las permutaciones de filas.
  # Retorna la matriz B y la permutación utilizada, o (None, None) si no se encuentra.

  n = A.shape[0]
  for perm in permutations(range(n)):
    B = A[list(perm)]
    if is_diagonally_dominant(B):
      return B, perm
  return None, None

def verify_decompositions(B, P, L, U, Q, R, U_svd, s, Vh):

  # Calcula la norma (error) entre la matriz original B y
  # la reconstrucción a partir de cada descomposición.
  error_lu  = np.linalg.norm(B - P @ L @ U)
  error_qr  = np.linalg.norm(B - Q @ R)
  error_svd = np.linalg.norm(B - U_svd @ np.diag(s) @ Vh)

  print("\n--- Verificación de descomposiciones ---")
  print("Las magnitudes indican la diferencia entre B y su reconstrucción.\nValores cercanos a 0 significan buena precisión.")
  print("Error LU (norma de B - P x L x U): ", error_lu)
  print("Error QR (norma de B - Q x R): ", error_qr)
  print("Error SVD (norma de B - U x diag(s) x Vh):", error_svd)


def main():
  # Lectura del orden de la matriz A y sus filas
  n = int(input("Ingrese el orden de la matriz cuadrada A: "))
  print("Ingrese cada fila separada por espacios:")
  rows = [list(map(float, input(f"Fila {i+1}: ").split())) for i in range(n)]
  A = np.array(rows)
  print("\nMatriz A:")
  print(A)

  # Verifica si A es diagonalmente dominante, si no, busca una permutación
  if is_diagonally_dominant(A):
    print("\nA es diagonalmente dominante.")
    B = A
  else:
    print("\nA no es diagonalmente dominante. Buscando permutación...")
    B, perm = make_diagonally_dominant(A)
    if B is None:
      print("No se encontró ninguna permutación que haga a A diagonalmente dominante.")
      return
    print("B obtenido con la permutación:", perm)
    print(B)

  # Factorización LU usando scipy.linalg.lu
  P, L, U = lu(B)
  print("\n--- Factorización LU ---")
  print("Matriz P (permutación):")
  print(P)
  print("Matriz L (triangular inferior):")
  print(L)
  print("Matriz U (triangular superior):")
  print(U)

  # Factorización QR usando numpy.linalg.qr
  Q, R = np.linalg.qr(B)
  print("\n--- Factorización QR ---")
  print("Matriz Q (ortogonal):")
  print(Q)
  print("Matriz R (triangular superior):")
  print(R)

  # Factorización SVD usando numpy.linalg.svd
  U_svd, s, Vh = np.linalg.svd(B)
  print("\n--- Factorización SVD ---")
  print("Matriz U:")
  print(U_svd)
  print("Valores singulares:")
  print(s)
  print("Matriz Vh:")
  print(Vh)

  # Verifica que las descomposiciones reconstruyen la matriz B
  verify_decompositions(B, P, L, U, Q, R, U_svd, s, Vh)

if __name__ == "__main__":
  main()

Ingrese el orden de la matriz cuadrada A: 3
Ingrese cada fila separada por espacios:
Fila 1: 2 2 6
Fila 2: 12 6 2
Fila 3: 1 4.5 2

Matriz A:
[[ 2.   2.   6. ]
 [12.   6.   2. ]
 [ 1.   4.5  2. ]]

A no es diagonalmente dominante. Buscando permutación...
B obtenido con la permutación: (1, 2, 0)
[[12.   6.   2. ]
 [ 1.   4.5  2. ]
 [ 2.   2.   6. ]]

--- Factorización LU ---
Matriz P (permutación):
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
Matriz L (triangular inferior):
[[1.         0.         0.        ]
 [0.08333333 1.         0.        ]
 [0.16666667 0.25       1.        ]]
Matriz U (triangular superior):
[[12.          6.          2.        ]
 [ 0.          4.          1.83333333]
 [ 0.          0.          5.20833333]]

--- Factorización QR ---
Matriz Q (ortogonal):
[[-0.9830783   0.11804024 -0.14008408]
 [-0.08192319 -0.9672742  -0.24014413]
 [-0.16384638 -0.22460435  0.96057652]]
Matriz R (triangular superior):
[[-12.20655562  -6.59481696  -3.1130813 ]
 [  0.          -4.09370117  -3.